In [1]:
# Need to find oil reading dates and exhaust gas reading date between the 2 work order dates, for each equipment
# then choose the latest reading date earlier than each work order date

In [1]:
import csv
import datetime

In [2]:
work_order_info_file = "work_order_info.csv"
work_order_dct = {}

with open(work_order_info_file) as work_order_csv:
    reader = csv.DictReader(work_order_csv)
    for r in reader:
        compid = r['compid']
        work_order_dct.setdefault(compid, set())
        work_order_dct[compid].add(r['DATE_WO'])

for compid, sch_date_lst in work_order_dct.items():
    print(compid, sch_date_lst)
    break

482 {'7/17/2017', '5/4/2017', '7/31/2017', '12/31/2016', '4/10/2017', '1/11/2017', '9/18/2016', '4/20/2017', '2/9/2017', '1/25/2017', '12/10/2016', '2/16/2017', '9/13/2016', '1/30/2017', '1/13/2017', '7/25/2017', '5/18/2017', '10/27/2016', '6/29/2017', '9/28/2016', '7/29/2017', '12/8/2016', '12/29/2016', '3/2/2017', '11/11/2016', '8/11/2016', '10/6/2016', '3/22/2017', '9/30/2016', '6/1/2017', '7/26/2016', '7/20/2016', '11/25/2016', '10/5/2016', '7/30/2017', '2/28/2017', '6/6/2017', '4/25/2017', '10/13/2016', '6/28/2017', '4/29/2017', '12/5/2016', '7/18/2017', '9/1/2016', '2/20/2017', '3/5/2017', '6/22/2017', '2/3/2017', '10/12/2016', '11/29/2016', '10/8/2016', '8/30/2016', '8/2/2016', '8/24/2016', '11/2/2016', '12/27/2016', '7/15/2017', '11/15/2016', '10/21/2016', '7/4/2017', '9/22/2016', '2/1/2017', '5/23/2017', '7/12/2016', '11/24/2016', '5/11/2017', '1/16/2017', '8/27/2016', '12/7/2016', '7/14/2016', '1/23/2017', '3/20/2017', '3/30/2017', '1/6/2017', '11/9/2016', '5/30/2017', '1/3/2

In [3]:
oil_analysis_file = "oil_analysis_data.csv"
oil_dct = {}

with open(oil_analysis_file) as oil_csv:
    reader = csv.DictReader(oil_csv)
    for r in reader:
        compid = r['EquipmentNumber']
        oil_dct.setdefault(compid, set())
        oil_dct[compid].add(r['sampledate'])
        
for compid, sample_date_lst in oil_dct.items():
    print(compid, sample_date_lst)
    break

341 {'13-Mar-17', '7-Jan-17', '12-Jan-17', '2-Aug-17', '6-Feb-17', '8-May-17', '15-Jul-17', '7-May-17', '22-Feb-17', '11-Jun-17', '1-Jul-17', '17-Apr-17', '30-Jan-17'}


In [39]:
exhaust_gas_file = "exhaust_gas_data.csv"
exhaust_gas_dct = {}

with open(exhaust_gas_file) as exhaust_gas_csv:
    reader = csv.DictReader(exhaust_gas_csv)
    for r in reader:
        comp_name = r['EquipmentName']
        compid = comp_name[2:]
        exhaust_gas_dct.setdefault(compid, {'comp_name':comp_name, 'readtime_lst':set()})
        exhaust_gas_dct[compid]['readtime_lst'].add(r['Readtime'])
        
for compid, subdct in exhaust_gas_dct.items():
    print(compid, subdct['comp_name'], list(subdct['readtime_lst'])[4:10])
    break

477 HT477 ['2016-12-07 20:44:56.520', '2016-12-07 17:07:46.507', '2016-12-07 20:28:43.233', '2016-11-06 17:57:26.297', '2016-07-20 07:56:20.793', '2016-07-20 08:19:17.867']


In [5]:
matched_dates = {}

for compid, sch_date_lst in work_order_dct.items():
    sorted_work_order_dates = list(sch_date_lst)
    sorted_work_order_dates.sort()
    
    if compid in oil_dct.keys():
        sorted_oil_dates = list(oil_dct[compid])
        sorted_oil_dates.sort()
    else:
        continue
    
#     if compid in exhaust_gas_dct.keys():
#         exhaust_gas_dates = list(exhaust_gas_dct[compid]['readtime_lst'])
#         exhaust_gas_dates.sort()
#     else:
#         continue
    
    for i in range(len(sorted_work_order_dates)):
        work_order_date = sorted_work_order_dates[i]
        if work_order_date is None or work_order_date == 'NA':
            continue
        work_order_date_time = datetime.datetime.strptime(work_order_date, '%m/%d/%Y')
            
        matched_dates.setdefault(compid,{work_order_date:''})
        
#         matched_dates.setdefault(compid,{'work_order_date':work_order_date, 'oil_date':'',
#                                          'exhaust_gas_date':'', 'comp_name':''})
            
        for j in range(len(sorted_oil_dates)):
            if sorted_oil_dates[j] is None or sorted_oil_dates[j]=='NA': 
                continue
            try:
                oil_date = datetime.datetime.strptime(sorted_oil_dates[j], '%d-%b-%y').strftime('%m/%d/%Y')
                oil_date = datetime.datetime.strptime(oil_date, '%m/%d/%Y')
                if (oil_date > work_order_date_time) == True:
                    continue
                if j == len(sorted_oil_dates)-1:
                    matched_dates[compid][work_order_date] = sorted_oil_dates[j]
                else: 
                    next_oil_date = datetime.datetime.strptime(sorted_oil_dates[j+1], '%d-%b-%y').strftime('%m/%d/%Y')
                    next_oil_date = datetime.datetime.strptime(next_oil_date, '%m/%d/%Y')
                    if (oil_date < work_order_date_time) == True and (next_oil_date > work_order_date_time) == True:
                        matched_dates[compid][work_order_date] = sorted_oil_dates[j]
                        break
            except:
                continue
                
#         for p in range(len(exhaust_gas_dates)):
#             if exhaust_gas_dates[p] is None or exhaust_gas_dates[p]=='NA': 
#                 continue
#             try:
#                 gas_date = datetime.datetime.strptime(exhaust_gas_dates[p].split()[0], '%Y-%m-%d').strftime('%m/%d/%Y')
#                 gas_date = datetime.datetime.strptime(gas_date, '%m/%d/%Y')
#                 if (gas_date > work_order_date_time) == True:
#                     continue
#                 if p == len(exhaust_gas_dates)-1:
#                     matched_dates[compid][work_order_date] = exhaust_gas_dates[p]
#                 else: 
#                     next_gas_date = datetime.datetime.strptime(exhaust_gas_dates[p+1].split()[0], '%Y-%m-%d').strftime('%m/%d/%Y')
#                     next_gas_date = datetime.datetime.strptime(next_gas_date, '%m/%d/%Y')
#                     if (gas_date < work_order_date_time) == True and (next_gas_date > work_order_date_time) == True:
#                         matched_dates[compid][work_order_date] = exhaust_gas_dates[j]
#                         break
#             except:
#                 continue

            
            
for compid, subdct in matched_dates.items():
    print(compid, subdct)
    break

482 {'1/11/2017': '', '1/24/2017': '23-Jan-17', '1/25/2017': '23-Jan-17', '1/26/2017': '23-Jan-17', '1/30/2017': '23-Jan-17', '2/1/2017': '23-Jan-17', '2/12/2017': '23-Jan-17', '2/16/2017': '23-Jan-17', '2/20/2017': '23-Jan-17', '2/28/2017': '22-Feb-17', '2/3/2017': '23-Jan-17', '2/5/2017': '23-Jan-17', '2/6/2017': '23-Jan-17', '2/7/2017': '23-Jan-17', '2/9/2017': '23-Jan-17', '3/12/2017': '22-Feb-17', '3/13/2017': '22-Feb-17', '3/14/2017': '22-Feb-17', '3/15/2017': '22-Feb-17', '3/2/2017': '22-Feb-17', '3/20/2017': '22-Feb-17', '3/21/2017': '22-Feb-17', '3/22/2017': '22-Feb-17', '3/26/2017': '22-Feb-17', '3/3/2017': '22-Feb-17', '3/30/2017': '22-Feb-17', '3/5/2017': '22-Feb-17', '4/10/2017': '22-Feb-17', '4/12/2017': '22-Feb-17', '4/16/2017': '22-Feb-17', '4/20/2017': '18-Apr-17', '4/25/2017': '18-Apr-17', '4/29/2017': '18-Apr-17', '4/3/2017': '22-Feb-17', '4/4/2017': '22-Feb-17', '5/10/2017': '18-Apr-17', '5/11/2017': '18-Apr-17', '5/17/2017': '14-May-17', '5/18/2017': '14-May-17', '

In [9]:
# write into csv

matched_time_out = "matched_time.csv"

with open(matched_time_out, 'w') as csv_file:
    fieldnames = ['compid', 'work_order_date', 'oil_sample_date']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for compid, subdct in matched_dates.items():
        for work_order_date, oil_date in subdct.items():
            if oil_date == '':
                continue
            writer.writerow({'compid':compid, 
                            'work_order_date':work_order_date, 
                            'oil_sample_date':oil_date})